In [0]:
%sh

# Download data and pre-trained model and train data
rm -rf /dbfs/home/TF/bert-large
mkdir -p /dbfs/home/TF/bert-large

mkdir -p /dbfs/home/TF/bert-large/SQuAD-1.1
cd /dbfs/home/TF/bert-large/SQuAD-1.1
wget https://github.com/zhixingheyi-tian/Resource/releases/download/BERT/dev-v1.1.json
wget https://github.com/zhixingheyi-tian/Resource/releases/download/BERT/evaluate-v1.1.py
wget https://github.com/zhixingheyi-tian/Resource/releases/download/BERT/train-v1.1.json
ls -l

cd /dbfs/home/TF/bert-large
wget https://storage.googleapis.com/intel-optimized-tensorflow/models/v1_8/bert_large_checkpoints.zip
unzip bert_large_checkpoints.zip

cd /dbfs/home/TF/bert-large
wget https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
unzip wwm_uncased_L-24_H-1024_A-16.zip
cd /dbfs/home/TF/bert-large/
ls -l
ls -l /dbfs/home/TF/bert-large/SQuAD-1.1



In [0]:
%sh

# BERT-Large Inference

# Install necessary package
sudo apt-get update
sudo apt-get install zip -y
sudo apt-get -y install git

# Remove old materials if exist
rm -rf /TF/
mkdir /TF/

# Create ckpt directory
mkdir -p /TF/BERT-Large-output/
export BERT_LARGE_OUTPUT=/TF/BERT-Large-output/

# Download IntelAI benchmark
cd /TF/
git clone https://github.com/IntelAI/models.git

export SQUAD_DIR=/dbfs/home/TF/bert-large/SQuAD-1.1/
export BERT_LARGE_DIR=/dbfs/home/TF/bert-large/

export PYTHONPATH=$PYTHONPATH:.

# Launch Benchmark for inference
cd /TF/models/benchmarks/
python3 launch_benchmark.py \
    --model-name=bert_large \
    --precision=fp32 \
    --mode=inference \
    --framework=tensorflow \
    --batch-size=32 \
    --data-location $BERT_LARGE_DIR/wwm_uncased_L-24_H-1024_A-16 \
    --checkpoint $BERT_LARGE_DIR/bert_large_checkpoints \
    --output-dir $BERT_LARGE_OUTPUT/bert-squad-output \
    --verbose \
    -- infer_option=SQuAD \
       DEBIAN_FRONTEND=noninteractive \
       predict_file=$SQUAD_DIR/dev-v1.1.json \
       experimental-gelu=False \
       init_checkpoint=model.ckpt-3649
  

In [0]:
%sh

# BERT-Large Training

# Install necessary package
sudo apt-get update
sudo apt-get install zip -y
sudo apt-get -y install git

# Remove old materials if exist
rm -rf /TF/
mkdir /TF/

# Create ckpt directory
mkdir -p /TF/BERT-Large-output/
export BERT_LARGE_OUTPUT=/TF/BERT-Large-output/

# Download IntelAI benchmark
cd /TF/
wget https://github.com/IntelAI/models/archive/refs/tags/v1.8.1.zip
unzip v1.8.1.zip
cd models-1.8.1/

export SQUAD_DIR=/dbfs/home/TF/bert-large/SQuAD-1.1
export BERT_LARGE_MODEL=/dbfs/home/TF/bert-large/wwm_uncased_L-24_H-1024_A-16

export PYTHONPATH=$PYTHONPATH:.

# Launch Benchmark for training
cd /TF/models-1.8.1/benchmarks/
python launch_benchmark.py \
    --model-name=bert_large \
    --precision=fp32 \
    --mode=training \
    --framework=tensorflow \
    --batch-size=4 \
    --benchmark-only \
    --data-location=$BERT_LARGE_MODEL \
    -- train-option=SQuAD  DEBIAN_FRONTEND=noninteractive   config_file=$BERT_LARGE_MODEL/bert_config.json init_checkpoint=$BERT_LARGE_MODEL/bert_model.ckpt     vocab_file=$BERT_LARGE_MODEL/vocab.txt train_file=$SQUAD_DIR/train-v1.1.json     predict_file=$SQUAD_DIR/dev-v1.1.json      do-train=True learning-rate=1.5e-5   max-seq-length=384     do_predict=True warmup-steps=0     num_train_epochs=0.1     doc_stride=128      do_lower_case=False     experimental-gelu=False     mpi_workers_sync_gradients=True
  

In [0]:
from __future__ import print_function
from tensorflow.python import _pywrap_util_port
import tensorflow
# Print version, and check whether is intel-optimized
print("tensorflow version: " +tensorflow.__version__)
print(_pywrap_util_port.IsMklEnabled())